# Cálculo Numérico – Interpolação Polinomial  
**Lista de Exercícios 10**

**Aluno:** Emanuel Rocha  
**Matrícula:** 0082834  
**Curso:** Engenharia de Computação  
**Instituição:** IFMG – Campus Bambuí  

---


## Objetivo

O objetivo deste trabalho é aplicar os métodos de **Interpolação Polinomial de Lagrange**
e de **Diferenças Divididas de Newton** para encontrar o polinômio interpolador de
conjuntos de pontos gerados aleatoriamente, conforme especificado na Lista de Exercícios 10.

Busca-se também verificar, na prática, a equivalência entre os dois métodos,
mostrando que ambos produzem o mesmo polinômio interpolador.


### Importação das bibliotecas

Nesta célula são importadas as bibliotecas necessárias para manipulação de dados,
leitura e escrita de arquivos CSV e operações numéricas.


In [7]:
import numpy as np
import csv

### Funções auxiliares

As funções a seguir são responsáveis por:
- gerar conjuntos de pontos aleatórios sem repetição das coordenadas `x`;
- salvar os pontos gerados em arquivos CSV;
- ler os pontos armazenados nos arquivos CSV para posterior processamento.


In [8]:
# funcoes auxiliares
def gera_pontos(semente):
    '''Gera pontos aleatórios sem repetir coordenada x'''
    np.random.seed(semente)
    num_pontos = np.random.randint(4, 8)
    n_min, n_max = -5 * num_pontos, 5 * num_pontos
    vet_x = np.random.choice(np.arange(n_min, n_max + 1), size=num_pontos, replace=False)
    vet_x.sort()
    vet_y = np.random.randint(n_min, n_max + 1, num_pontos)
    return np.column_stack((vet_x, vet_y))

# Função para salvar os pontos em um arquivo CSV
def salva_csv(pontos, nome_arquivo):
    with open(nome_arquivo, 'w', newline='', encoding='utf8') as arq_csv:
        writer = csv.writer(arq_csv)
        for row in pontos:
            writer.writerow(row)

# Função para ler pontos de um arquivo CSV
def ler_pontos_csv(nome_arquivo):
    x, y = [], []
    with open(nome_arquivo, encoding='utf8') as arq_csv:
        reader = csv.reader(arq_csv)
        for row in reader:
            x.append(float(row[0]))
            y.append(float(row[1]))
    return x, y

### Métodos de interpolação polinomial

Nesta célula são implementados os métodos de:
- **Diferenças Divididas de Newton**;
- **Interpolação de Lagrange**,

utilizados para a construção do polinômio interpolador a partir dos pontos fornecidos.


In [9]:
# Função para interpolação de Newton
def divididas_newton(x, y):
    num_pontos = len(x)
    mat_t = np.zeros((num_pontos, num_pontos))
    mat_t[:, 0] = y

    # Construção da tabela de diferenças divididas
    for col in range(1, num_pontos):
        for lin in range(num_pontos - col):
            mat_t[lin, col] = (mat_t[lin + 1, col - 1] - mat_t[lin, col - 1]) / (x[lin + col] - x[lin])

    # Construção do polinômio interpolador
    poli_final = np.poly1d(0)
    termo = np.poly1d(1)
    for i in range(num_pontos):
        if i > 0:
            termo = np.polymul(termo, np.poly1d([1, -x[i - 1]]))
        poli_final += mat_t[0, i] * termo

    return poli_final

# Função para interpolação de Lagrange
def lagrange(x, y):
    num_pontos = len(x)
    poli_final = np.poly1d(0)

    for i in range(num_pontos):
        num = np.poly1d(1)
        den = 1
        for j in range(num_pontos):
            if i != j:
                num = np.polymul(num, np.poly1d([1, -x[j]]))
                den *= (x[i] - x[j])
        poli_final += (y[i] / den) * num

    return poli_final

### Geração dos conjuntos de pontos

Nesta etapa são definidos os valores das sementes a partir da matrícula do aluno.
Para cada semente, é gerado um conjunto de pontos aleatórios, que é salvo
em um arquivo CSV distinto.


In [10]:
# Geração dos arquivos CSV com pontos aleatórios
matricula = 82834
sementes = [matricula + i * 100 for i in range(5)]

arquivos_csv = []
for semente in sementes:
    pontos = gera_pontos(semente)
    nome_arquivo = f"{semente}.csv"
    salva_csv(pontos, nome_arquivo)
    arquivos_csv.append(nome_arquivo)

# Aplicar os métodos de interpolação nos arquivos gerados
resultados = {}
for arquivo in arquivos_csv:
    x, y = ler_pontos_csv(arquivo)
    poli_newton = divididas_newton(x, y)
    poli_lagrange = lagrange(x, y)
    resultados[arquivo] = {"Newton": poli_newton, "Lagrange": poli_lagrange}

### Exibição dos resultados

Nesta célula são exibidos os polinômios interpoladores obtidos por ambos
os métodos, permitindo a comparação direta entre os resultados.


In [11]:
# Exibir os polinômios gerados
for arquivo, polinomios in resultados.items():
    print(f"\nArquivo: {arquivo}")
    print("Polinômio de Newton:")
    print(polinomios["Newton"])
    print("\nPolinômio de Lagrange:")
    print(polinomios["Lagrange"])
    print("-" * 50)



Arquivo: 82834.csv
Polinômio de Newton:
            6             5            4           3          2
-2.065e-06 x - 5.501e-05 x + 0.002375 x + 0.06187 x - 0.3966 x - 8.595 x + 29.93

Polinômio de Lagrange:
            6             5            4           3          2
-2.065e-06 x - 5.501e-05 x + 0.002375 x + 0.06187 x - 0.3966 x - 8.595 x + 29.93
--------------------------------------------------

Arquivo: 82934.csv
Polinômio de Newton:
            5             4            3           2
-8.826e-06 x - 0.0001502 x + 0.009748 x + 0.06191 x - 2.012 x + 0.8078

Polinômio de Lagrange:
            5             4            3           2
-8.826e-06 x - 0.0001502 x + 0.009748 x + 0.06191 x - 2.012 x + 0.8078
--------------------------------------------------

Arquivo: 83034.csv
Polinômio de Newton:
           3          2
-0.008923 x + 0.4524 x - 4.464 x + 0.1906

Polinômio de Lagrange:
           3          2
-0.008923 x + 0.4524 x - 4.464 x + 0.1906
---------------------------------